In [ ]:
import numpy as np
import pandas as pd

# Import matplotlib, seaborn for visualization
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#weather_data["TIMESTAMP"] = pd.to_datetime(weather_data["TIMESTAMP"], format='%d-%m-%Y').dt.strftime('%Y-%m-%d')

In [ ]:
btnt=pd.read_csv('new.csv',parse_dates=['TIMESTAMP']
                     , infer_datetime_format=True)

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
#a=weather_data['TIMESTAMP']
btnt.iloc[:,1:] = imputer.fit_transform(btnt.iloc[:,1:])

In [ ]:
#additive decomposition
from statsmodels.tsa.seasonal import seasonal_decompose

result_add = seasonal_decompose(btnt.WindSpeed_ms_Max, model='additive', extrapolate_trend='freq', freq=365)

plt.rcParams.update({'figure.figsize': (10,10)})
result_add.plot().suptitle('Additive Decomposition', fontsize=22)
plt.show()

In [ ]:
#shifting
predicted_df = btnt["WindSpeed_ms_Max"].to_frame().shift(1).rename(columns = {"WindSpeed_ms_Max": "WindSpeed_ms_Max_pred" })
actual_df = btnt["WindSpeed_ms_Max"].to_frame().rename(columns = {"WindSpeed_ms_Max": "WindSpeed_ms_Maxactual" })
one_step_df = pd.concat([actual_df,predicted_df],axis=1)
one_step_df = one_step_df[1:]
one_step_df.head(10)

In [ ]:
import itertools
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
import warnings
warnings.filterwarnings("ignore")

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(one_step_df.AmbTemp_C_Avgactual,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('SARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
import statsmodels.api as sm
mod = sm.tsa.statespace.SARIMAX(one_step_df.WindSpeed_ms_Maxactual,
                                order=(1, 1, 0),
                                seasonal_order=(1, 1, 0, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:

pred = results.predict(start=600,end=822)[1:].rename('data sarimax (1,1,0) forecast')
#pred_ci = pred.conf_int()

In [ ]:
ax=one_step_df.WindSpeed_ms_Maxactual.plot(figsize=(12,8),legend=True,label='Actual max wind speed output')
pred.plot(legend=True)
ax.set_xlabel('Number of days')
ax.set_ylabel('Max wind speed(m/s)')